In [50]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/caltech256")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/caltech256


In [51]:
from pathlib import Path

from torchvision.models import resnet18, efficientnet_b0

root_dir = path + '/256_ObjectCategories'

def get_folder_frequency_list(root_dir):
    folder_frequency_list = []

    root_path = Path(root_dir)

    for folder in sorted(root_path.iterdir()):
        if folder.is_dir():
            file_count = sum(1 for item in folder.iterdir() if item.is_file())
            folder_frequency_list.extend([folder.name.split('.')[0]] * file_count)

    return folder_frequency_list

targets = get_folder_frequency_list(root_dir)
targets_int = [int(x) for i,x in enumerate(targets)]


In [52]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

root_dir = '/kaggle/input/'

transform1 = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.Lambda(lambda img: img.convert("RGB")),
    # transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

transform2 = transforms.Compose([
    transforms.Resize((256, 256)),
    # transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

inception_dataset = datasets.Caltech256(root=root_dir, transform = transform1, target_transform = None,  download=False)
resnet_dataset = datasets.Caltech256(root=root_dir, transform=transform2, target_transform=None, download= False)

In [53]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch

indices = np.arange(len(inception_dataset))

if len(indices) != len(targets_int):
    del targets_int[0]
    targets_int = np.array(targets_int)

train_indices, temp_indices, _, _ = train_test_split(indices, targets_int, test_size=0.3, stratify=targets_int, random_state=42)

val_indices, test_indices, _, _ = train_test_split(temp_indices, [targets_int[i] for i in temp_indices], test_size=0.5, stratify=[targets_int[i] for i in temp_indices], random_state=42)

train_dataset = torch.utils.data.Subset(inception_dataset, train_indices)
val_dataset = torch.utils.data.Subset(inception_dataset, val_indices)
test_dataset = torch.utils.data.Subset(inception_dataset, test_indices)


print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Train size: 21424
Validation size: 4591
Test size: 4592


In [54]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch

indices = np.arange(len(resnet_dataset))

if len(indices) != len(targets_int):
    del targets_int[0]
    targets_int = np.array(targets_int)

train_indices, temp_indices, _, _ = train_test_split(indices, targets_int, test_size=0.3, stratify=targets_int, random_state=42)

val_indices, test_indices, _, _ = train_test_split(temp_indices, [targets_int[i] for i in temp_indices], test_size=0.5, stratify=[targets_int[i] for i in temp_indices], random_state=42)

train_dataset = torch.utils.data.Subset(resnet_dataset, train_indices)
val_dataset = torch.utils.data.Subset(resnet_dataset, val_indices)
test_dataset = torch.utils.data.Subset(resnet_dataset, test_indices)

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

resnet_train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
resnet_val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)
resnet_test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

Train size: 21424
Validation size: 4591
Test size: 4592


In [26]:
def topk_accuracy(model, loader, device, k=5):
    
    model.eval()
    correct_topk = 0
    total = 0

    with torch.no_grad():

        for images, labels in loader:
            
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, topk_preds = outputs.topk(k, dim=1) 

            correct_topk += sum([label in topk for topk, label in zip(topk_preds, labels)])
            total += labels.size(0)

    return correct_topk / total


In [55]:
import torch
import torch.nn as nn

class ResNetModule(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):

        super(ResNetModule, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
      
        identity = x

        pred = self.relu(self.bn1(self.conv1(x)))
        pred = self.relu(self.bn2(self.conv2(pred)))
        pred = self.bn3(self.conv3(pred))

        if self.downsample:
            identity = self.downsample(x)

        pred += identity
        pred = self.relu(pred)

        return pred






class myResNet50(nn.Module):

    def __init__(self):
        
        super(myResNet50, self).__init__()

        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self.make_layer(ResNetModule, 64, 3)
        self.layer2 = self.make_layer(ResNetModule, 128, 4, stride=2)
        self.layer3 = self.make_layer(ResNetModule, 256, 6, stride=2)
        self.layer4 = self.make_layer(ResNetModule, 512, 3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * ResNetModule.expansion, 512)
        
    


    def make_layer(self, block, out_channels, blocks, stride=1):  #Some help taken from ai here

        downsample = None

        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = [block(self.in_channels, out_channels, stride, downsample)]    
        self.in_channels = out_channels * block.expansion

        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):

        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [56]:
import torch.nn.functional as F

class InceptionModule(nn.Module):

    def __init__(self, in_channels, b1, b2a, b2b, b3a, b3b, pool_proj):

        super(InceptionModule, self).__init__()

        self.branch1 = nn.Conv2d(in_channels, b1, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, b2a, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(b2a, b2b, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, b3a, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(b3a, b3b, kernel_size=5, padding=2),
            nn.ReLU()
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1),
            nn.ReLU()
        )

    def forward(self, x):

        return torch.cat([
            self.branch1(x),
            self.branch2(x),
            self.branch3(x),
            self.branch4(x)
        ], 1)





class myInceptionV3(nn.Module):

    def __init__(self, num_classes=257):

        super(myInceptionV3, self).__init__()
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2)
        )

        self.inception3a = InceptionModule(64, 64, 48, 64, 64, 96, 32)
        self.inception3b = InceptionModule(256, 64, 48, 64, 64, 96, 64)

        self.maxpool = nn.MaxPool2d(3, stride=2)

        self.inception4a = InceptionModule(288, 64, 48, 64, 64, 96, 64)
        self.inception4b = InceptionModule(288, 64, 48, 64, 64, 96, 64)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(288, num_classes)


    def forward(self, x):

        x = self.pre_layers(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)   #This was suggested by ai
        x = self.dropout(x)
        x = self.fc(x)

        return x

In [57]:
import torchvision.models as models
import torch.optim as optim

class Ensembler:

    def __init__(self, model1, model2):
        
        pre1 = models.resnet50(pretrained = True)
        self.my1 = model1

        my_w = self.my1.state_dict()
        pre_w = pre1.state_dict()

        matched = {}

        for i, j in pre_w.items():
            if i in my_w and j.shape == my_w[i].shape:
                matched[i] = j

        my_w.update(matched)
        self.my1.load_state_dict(my_w)
        
        torch.save(self.my1.state_dict(), "resnet50.pth")
        self.my1.load_state_dict(torch.load("resnet50.pth"))

        
        pre2 = models.inception_v3(pretrained = True)
        self.my2 = model2

        my_w = self.my2.state_dict()
        pre_w = pre2.state_dict()

        matched = {}

        for i, j in pre_w.items():
            if i in my_w and j.shape == my_w[i].shape:
                matched[i] = j

        my_w.update(matched)
        self.my2.load_state_dict(my_w)

        torch.save(self.my2.state_dict(), "inception_v3.pth")
        self.my2.load_state_dict(torch.load("inception_v3.pth"))


        self.criterion = nn.CrossEntropyLoss()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def optimizers(self, mode):
        return optim.Adam(filter(lambda p: p.requires_grad, mode.parameters()), lr=1e-4)

    def fine_tune(self, model, dataloader, criterion, epochs, device):

        for params in model.parameters():
            params.require_grad = False
        for params in model.fc.parameters():
            params.require_grad = True

        optimizer = self.optimizers(model)
        
        for epoch in range(epochs):
            model.train()
            total_loss = 0
            
            for images, labels in dataloader:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            total_loss += loss.item()

            print(f"Epoch:{epoch+1}, Loss:{total_loss}")


    def topk_performance(self, model, loader, device, k):
    
            model.eval()
            correct_topk = 0
            total = 0
    
            with torch.no_grad():
    
                for images, labels in loader:
                
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    _, topk_preds = outputs.topk(k, dim=1) 
    
                    correct_topk += sum([label in topk for topk, label in zip(topk_preds, labels)])
                    total += labels.size(0)
    
            return correct_topk / total
            

    def fine_tune_ensembler(self, train_resnet, train_inception):

        print("Fine tuning resnet50.")
        self.fine_tune(self.my1, train_resnet, self.criterion, 20, self.device)

        print("Fine tuning inception_v3.")
        self.fine_tune(self.my2, train_inception, self.criterion, 20, self.device)

    
    

In [58]:
device = 'cuda'

model1 = myResNet50().to(device)
model2 = myInceptionV3().to(device)

ensemble = Ensembler(model1, model2)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-d

In [59]:
ensemble.fine_tune_ensembler(resnet_train_loader, train_loader)

Fine tuning resnet50.
Epoch:1, Loss:1.647470474243164
Epoch:2, Loss:1.31736421585083
Epoch:3, Loss:0.24041448533535004
Epoch:4, Loss:0.1425720602273941
Epoch:5, Loss:0.06755145639181137
Epoch:6, Loss:0.028846291825175285
Epoch:7, Loss:0.011470510624349117
Epoch:8, Loss:0.005069027189165354
Epoch:9, Loss:0.011143187992274761
Epoch:10, Loss:0.004602707922458649
Epoch:11, Loss:0.005529246758669615
Epoch:12, Loss:0.006758340168744326
Epoch:13, Loss:0.030282966792583466
Epoch:14, Loss:0.35267373919487
Epoch:15, Loss:0.14541758596897125
Epoch:16, Loss:0.01237315684556961
Epoch:17, Loss:0.008470219559967518
Epoch:18, Loss:0.0029532008338719606
Epoch:19, Loss:0.0016067521646618843
Epoch:20, Loss:0.003948498982936144
Fine tuning inception_v3.
Epoch:1, Loss:5.377089500427246
Epoch:2, Loss:5.356916427612305
Epoch:3, Loss:4.549125671386719
Epoch:4, Loss:5.200327396392822
Epoch:5, Loss:4.8500847816467285
Epoch:6, Loss:4.796204566955566
Epoch:7, Loss:4.51137638092041
Epoch:8, Loss:4.774440288543701


In [60]:
ensemble.topk_performance(ensemble.my1, resnet_train_loader, device, 5)


1.0

In [61]:
ensemble.topk_performance(ensemble.my2, train_loader, device, 5)

0.42564413741598206